In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time


In [3]:
df=pd.read_excel(r"../dataset/data.xlsx",sheet_name="Sheet1")

df

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350
...,...,...,...,...,...,...,...
4021,22,66,6,6.0,50,25.00,1.628000
4022,22,66,6,6.0,75,25.00,2.714000
4023,22,66,6,6.0,100,25.00,2.879000
4024,22,66,6,7.5,0,25.00,0.602680


In [4]:
df.shape

(4026, 7)

In [5]:
df.head()

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350


In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)

normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

print(normalized_df)

      Number of Storeys(NS)  Height of Structure(HS)  Number of Spans(HSP)  \
0                       0.0                      0.0                   0.0   
1                       0.0                      0.0                   0.0   
2                       0.0                      0.0                   0.0   
3                       0.0                      0.0                   0.0   
4                       0.0                      0.0                   0.0   
...                     ...                      ...                   ...   
4021                    1.0                      1.0                   1.0   
4022                    1.0                      1.0                   1.0   
4023                    1.0                      1.0                   1.0   
4024                    1.0                      1.0                   1.0   
4025                    1.0                      1.0                   1.0   

      Length of Spans (LS)  Opening percentage (OP)  \
0       

In [8]:
normalized_df.keys()

Index(['Number of Storeys(NS)', 'Height of Structure(HS)',
       'Number of Spans(HSP)', 'Length of Spans (LS)',
       'Opening percentage (OP)', 'Masonry wall Stiffness Et (MS)',
       'Period (P)'],
      dtype='object')

In [9]:
X=normalized_df.drop("Period (P)" ,axis=1)

In [10]:
y=normalized_df["Period (P)"]

In [12]:
from sklearn.model_selection import train_test_split, KFold
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=99)

In [15]:

from sklearn.model_selection import KFold,cross_validate
from bayes_opt import BayesianOptimization
from hyperopt import hp,fmin,tpe,Trials,partial
from hyperopt.early_stop import no_progress_loss
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
import optuna.integration.lightgbm as oplgb
from catboost import CatBoostRegressor
from bayes_opt import BayesianOptimization

In [16]:

def bayesopt_objective(iterations,depth,l2_leaf_reg,learning_rate):
    model=CatBoostRegressor(iterations=int(iterations)
              ,depth=int(depth)
              ,l2_leaf_reg=int(l2_leaf_reg)
              ,learning_rate=learning_rate
              )
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                   ,error_score='raise')
    
    return np.mean(abs(validate_loss['test_score']))

In [18]:
def param_bayes_opt(init_points,n_iter):
    opt=BayesianOptimization(bayesopt_objective
                             ,param_grid_simple
                             ,random_state=99)
    
        opt.maximize(init_points=init_points 
                 ,n_iter=n_iter 
                )
    
   
    params_best=opt.max['params'] 
    score_best=opt.max['target'] 
    
        print("\n","best params: ", params_best,
          "\n","best cvscore: ", score_best)
    
    return params_best,score_best

In [19]:
def bayes_opt_validation(params_best):
    model=CatBoostRegressor(iterations=int(params_best['iterations'])
            ,depth=int(params_best['depth'])
            ,l2_leaf_reg=int(params_best['l2_leaf_reg'])
            ,learning_rate=params_best['learning_rate']
           )
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                  )
    
    return np.mean(abs(validate_loss['test_score']))

In [20]:

start_time = time.time()

param_grid_simple={'iterations':(100,1500)
                  ,'depth':(3,10)
                  ,'l2_leaf_reg':(0.01,10)
                   ,'learning_rate':( 0.01,0.2)
                  }

params_best,score_best=param_bayes_opt(20,100)

params_best 
score_best 
validation_score=bayes_opt_validation(params_best)
validation_score # day
# End time
end_time = time.time()
# Calculate total time
total_time = end_time - start_time
print("Total time taken: ", total_time)

|   iter    |  target   |   depth   | iterat... | l2_lea... | learni... |
-------------------------------------------------------------------------
| 1         | 0.008373  | 7.706     | 783.3     | 8.257     | 0.01597   |
| 2         | 0.005103  | 8.656     | 891.9     | 2.983     | 0.01887   |
| 3         | 0.007862  | 9.934     | 109.6     | 7.7       | 0.1519    |
| 4         | 0.004548  | 5.642     | 791.8     | 9.29      | 0.08514   |
| 5         | 0.004412  | 9.818     | 834.2     | 0.9452    | 0.1645    |
| 6         | 0.004385  | 4.482     | 876.1     | 2.93      | 0.1651    |
| 7         | 0.008033  | 8.796     | 410.2     | 6.452     | 0.02808   |
| 8         | 0.008685  | 5.882     | 235.6     | 1.449     | 0.05032   |
| 9         | 0.04137   | 6.337     | 208.7     | 2.358     | 0.01125   |
| 10        | 0.004326  | 9.291     | 873.1     | 1.684     | 0.1865    |
| 11        | 0.006644  | 6.795     | 158.5     | 5.254     | 0.1316    |
| 12        | 0.004005  | 8.605     | 

| 67        | 0.005462  | 5.634     | 207.5     | 2.99      | 0.1982    |
| 68        | 0.01078   | 4.432     | 152.9     | 1.173     | 0.09867   |
| 69        | 0.005352  | 6.19      | 237.3     | 3.031     | 0.1096    |
| 70        | 0.04422   | 3.376     | 153.8     | 1.923     | 0.0244    |
| 71        | 0.003239  | 6.775     | 1.352e+03 | 1.77      | 0.1884    |
| 72        | 0.005576  | 6.35      | 208.1     | 2.578     | 0.1023    |
| 73        | 0.004488  | 4.324     | 927.5     | 0.2968    | 0.1193    |
| 74        | 0.004417  | 6.356     | 570.6     | 2.887     | 0.05681   |
| 75        | 0.008117  | 5.142     | 165.9     | 6.454     | 0.1029    |
| 76        | 0.003575  | 7.962     | 1.012e+03 | 4.588     | 0.1814    |
| 77        | 0.008276  | 3.755     | 726.7     | 9.802     | 0.1405    |
| 78        | 0.007483  | 3.582     | 1.176e+03 | 8.696     | 0.1129    |
| 79        | 0.003849  | 5.04      | 1.434e+03 | 5.462     | 0.0615    |
| 80        | 0.00793   | 7.975     | 

# CatBoost

In [21]:
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [22]:
from catboost import CatBoostRegressor
params_best['depth'] = int(params_best['depth'])
params_best['iterations'] = int(params_best['iterations'])
cat_reg = CatBoostRegressor(**params_best )
cat_reg.fit(X_train, y_train)


0:	learn: 0.2171031	total: 142ms	remaining: 21.6s
1:	learn: 0.2132324	total: 143ms	remaining: 10.8s
2:	learn: 0.2094239	total: 144ms	remaining: 7.19s
3:	learn: 0.2057905	total: 145ms	remaining: 5.39s
4:	learn: 0.2020487	total: 145ms	remaining: 4.3s
5:	learn: 0.1985591	total: 146ms	remaining: 3.58s
6:	learn: 0.1950614	total: 147ms	remaining: 3.06s
7:	learn: 0.1918195	total: 147ms	remaining: 2.67s
8:	learn: 0.1883812	total: 149ms	remaining: 2.38s
9:	learn: 0.1851342	total: 151ms	remaining: 2.16s
10:	learn: 0.1822735	total: 152ms	remaining: 1.96s
11:	learn: 0.1791452	total: 153ms	remaining: 1.79s
12:	learn: 0.1761730	total: 153ms	remaining: 1.65s
13:	learn: 0.1734205	total: 154ms	remaining: 1.53s
14:	learn: 0.1703928	total: 154ms	remaining: 1.42s
15:	learn: 0.1674516	total: 155ms	remaining: 1.33s
16:	learn: 0.1646712	total: 156ms	remaining: 1.25s
17:	learn: 0.1619483	total: 157ms	remaining: 1.17s
18:	learn: 0.1593614	total: 157ms	remaining: 1.11s
19:	learn: 0.1568552	total: 158ms	remainin

In [29]:
y_pred= cat_reg.predict(X_test)
y_pred1=cat_reg.predict(X_val)
y_pred2=cat_reg.predict(X_train)

In [30]:
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [31]:
print("mean_absolute_error:", mean_absolute_error(y_val, y_pred1))
print("mean_squared_error:", mean_squared_error(y_val, y_pred1))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred1)))
print("r2 score:", r2_score(y_val, y_pred1))

mean_absolute_error: 0.03330253608556198
mean_squared_error: 0.002204713409074028
rmse: 0.04695437582456004
r2 score: 0.9557808657808284


In [32]:
print("mean_absolute_error:", mean_absolute_error(y_train, y_pred2))
print("mean_squared_error:", mean_squared_error(y_train, y_pred2))
print("rmse:", sqrt(mean_squared_error(y_train, y_pred2)))
print("r2 score:", r2_score(y_train, y_pred2))

mean_absolute_error: 0.031992328042325796
mean_squared_error: 0.0018954240684203093
rmse: 0.04353646825846476
r2 score: 0.9612508918196697


In [33]:
print("mean_absolute_error:", mean_absolute_error(y_test, y_pred))
print("mean_squared_error:", mean_squared_error(y_test, y_pred))
print("rmse:", sqrt(mean_squared_error(y_test, y_pred)))
print("r2 score:", r2_score(y_test, y_pred))

mean_absolute_error: 0.03229026680144465
mean_squared_error: 0.0019288162738250948
rmse: 0.04391829088005469
r2 score: 0.9623503554723772
